<h1> Face Anonymizer </h1>

<h2> imports </h2>

In [1]:
# !pip install mediapipe==0.9.3.0
# !pip install --upgrade protobuf

In [1]:
import cv2
import mediapipe as mp
import argparse

<h2> detect faces and blur </h2>

In [2]:
def process_img(img, face_detection):
    # Get image dimensions
    H, W, _ = img.shape
    # Convert image to RGB format (required by MediaPipe)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Run face detection on the image
    out = face_detection.process(img_rgb)
    
    # Check if faces are detected
    if out.detections is not None:
        # Loop through each detected face
        for detection in out.detections:
            location_data = detection.location_data
            bbox = location_data.relative_bounding_box

            # Extract bounding box coordinates and scale to image dimensions
            x1, y1, w, h = bbox.xmin, bbox.ymin ,bbox.width, bbox.height
            x1 = int(x1 * W)
            y1 = int(y1 * H)
            w = int(w * W)
            h = int(h * H)
            
            
            
            # Apply blur to the region of the face
            img[y1:y1 + h, x1:x1 + w, :] = cv2.blur(img[y1:y1 + h, x1:x1 + w, :], (30, 30))
            
            # Draw a green rectangle around the detected face
            img = cv2.rectangle(img, (x1, y1), (x1 + w, y1 + h), (0, 255, 0), 10)
    
    return img

In [3]:
# Parse command line arguments
args = argparse.ArgumentParser()
args.add_argument('--mode', default='video')
args.add_argument('--filePath', default='testVideo.mp4')
args = args.parse_args(args=[])

# Initialize MediaPipe face detection module
mp_face_detection = mp.solutions.face_detection

with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5) as face_detection:
    # Choose mode of operation
    if args.mode in ['image']:
        img = cv2.imread(args.filePath)
        img = process_img(img, face_detection)
        cv2.imshow('img', img)
        cv2.waitKey(0)
        
    elif args.mode in ['video']:
        cap = cv2.VideoCapture(args.filePath)
        ret, frame = cap.read()
        # Initialize video writer for output
        output_video = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 25, (frame.shape[1], frame.shape[0]))
        
        while ret:
            frame = process_img(frame, face_detection)
            # Write processed frame to output video
            output_video.write(frame)
            ret, frame = cap.read()
            
        cap.release()
        output_video.release()
        
    elif args.mode in ['webcam']:
        cap = cv2.VideoCapture(1)
        ret, frame = cap.read()
        
        while ret:
            frame = process_img(frame, face_detection)
            # Display the processed frame
            cv2.imshow('frame', frame)
            # Wait for 25 milliseconds (adjust as needed)
            cv2.waitKey(25)
            ret, frame = cap.read()
        
        cap.release()
        cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


<h2> blur </h2>